<a href="https://colab.research.google.com/github/Midmost/AI/blob/master/EDA_gu_gangseo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install --upgrade pandas

## Basic information

In [ ]:
import pandas as pd
import numpy as np
from pandas.io.formats.info import DataFrameInfo


# excel path
xlsx_file_path = "/content/drive/MyDrive/DATA/pdf2xlsx/editXL/gangseo.xlsx"

# pure excel to pure dataframe and information with a visual-table
df = pd.read_excel(xlsx_file_path)
DataFrameInfo
df

* ***하지만 만약 xlsx 파일 혹은 pdf 파일을 확인하고 내용이 다르다면 그에 맞춰 header를 수정해야 할 수 있음*** 

* 헤더의 명칭은 각 구의 계층구조에 정확히 맞춰서 수정하였었으나 원래 주셨던 대로 열 수를 맞춤

In [ ]:
!pip install --upgrade pandas

In [ ]:
xlsx_file_path = "/content/drive/MyDrive/DATA/pdf2xlsx/editXL/gangseo.xlsx"

import pandas as pd
import numpy as np

# df 내용 이쁘게 확인하기
from pandas.io.formats.info import DataFrameInfo
df = pd.read_excel(xlsx_file_path)
DataFrameInfo


df = pd.read_excel(xlsx_file_path, header = 3)

# Add a new column to the first index of the dataframe
df.insert(0, '구', '강서구')
header_all = "구,부서,정책,단위,세부사업,편성목,편성운영비,편성세부운영비,편성세부내역비용,예산액,전년도예산액,비교증감"
header = header_all.split(",")
df.columns = header

columns_to_fill = ['부서', '정책', '단위', '세부사업', '편성목', '편성운영비']
# columns_to_fill = header[:5]
for col in columns_to_fill:
    df[col] = df[col].fillna(method='ffill')

In [ ]:
# 그냥 원래 예시2에 맞춘거...
df['편성세부운영비'] = df['편성세부운영비'].astype(str).replace('nan', '')
df['편성세부내역비용'] = df['편성세부내역비용'].astype(str).replace('nan', '')
df['편성세부운영비'] = df['편성세부운영비'] + df['편성세부내역비용']
df['편성세부운영비']
df = df.drop(columns=['편성세부내역비용'])


In [ ]:
df

1. If a row with index i has a value in the “편성세부운영비” column that starts with “1)” and the next row starts with “2)”, do not merge the rows.


2. If a row with index i has a value in the “편성세부운영비” column that starts with “2)” and the next row starts with “3)”, do not merge the rows.


3. Otherwise, merge row i with row i+1.


4. When merging two rows, do not allow duplicate values.
---
1. i라는 인덱스를 가진 row이면서 열이 "편성세부운영비" 인 경우이면서 "1)"로 시작 할 때 아래 행이 "2)"로 시작하면 pass
2. i라는 인덱스를 row이면서 열이 "편성세부운영비"인 경우이면서 "2)"로 시작 할 때 아래 행이 "3)"이면 pass
3. 아니라면 i 라는 인덱스를 가진 row는 i+1 인덱스를 가진 row와 병합을 한다. 
4.  이 때 합치려는 두 셀의 값이 같은 경우에는 중복은 허용하지 않는다.   


In [ ]:

# Extract rows where the value in the 'A' column starts with '1)' and the value in the next row starts with '2)'
# mask = ((df['편성세부운영비'].str.startswith('1)')) & (df['편성세부운영비'].shift(1).str.startswith('2)'))) | ((df['편성세부운영비'].str.startswith('2)')) & (df['편성세부운영비'].shift(1).str.startswith('3)'))) | (((df['편성세부운영비'].str.startswith('1)')) & (df['편성세부운영비'].shift(1).str.startswith(''))))
# same with this
# Create a boolean mask
mask = (
    # Select rows where the value in the '편성세부운영비' column starts with '1)' and the value in the previous row starts with '2)'
    ((df['편성세부운영비'].str.startswith('1)')) & (df['편성세부운영비'].shift(1).str.startswith('2)'))) |
    # Select rows where the value in the '편성세부운영비' column starts with '2)' and the value in the previous row starts with '3)'
    ((df['편성세부운영비'].str.startswith('2)')) & (df['편성세부운영비'].shift(1).str.startswith('3)'))) |
    # Select rows where the value in the '편성세부운영비' column starts with '1)' and the value in the previous row is an empty string
    ((df['편성세부운영비'].str.startswith('1)')) & (df['편성세부운영비'].shift(1).str.startswith('')))
)




# Invert the boolean values in the mask
inverse_mask = ~mask



df['mask'] = df['편성세부운영비'].shift(1)

df['mask'] = mask

# Iterate over the rows of the DataFrame
for i in range(len(df)-1):
    # Check if the value in the current row of the 'mask' column is True
    if df.at[i, 'mask']:
        # Combine the values in the current and next rows of the '편성세부운영비' column
        combined_value = df.at[i, '편성세부운영비'] + ' ' + df.at[i+1, '편성세부운영비']
        # Assign the combined value to the current row of the 'combined' column
        df.at[i, 'combined'] = combined_value


In [ ]:
# Check the number of rows in the DataFrame
num_rows = df.shape[0]
print(f'The DataFrame has {num_rows} rows.')

In [ ]:
df

In [ ]:

# Define a function to check if a row should be merged with the next row
def should_merge():
    if df['편성세부운영비'].startswith('1)') and df['편성세부운영비'].startswith('2)'):
        return False
    elif df['편성세부운영비'].startswith('2)') and df['편성세부운영비'].startswith('3)'):
        return False
    else:
        return True

# Create a new column that contains the value of the "편성세부운영비" column from the next row
df['next'] = df['편성세부운영비'].shift(-1)

# Apply the should_merge function to each row of the DataFrame
df['merge'] = df.apply(should_merge, axis=1)

# Merge rows based on the value of the "merge" column
result = []
for i, row in df.iterrows():
    if row['merge']:
        result.append(row['편성세부운영비'] + ' ' + row['next'])
    else:
        result.append(row['편성세부운영비'])

# Create a new DataFrame from the result
result = pd.DataFrame({'merged': result})

# Display the resulting DataFrame
print(result)

In [ ]:
df.iloc[15404]

In [ ]:
# Export the DataFrame to an Excel file
df.to_excel('output3.xlsx', index=False)